# Pasos:

1. Particionar datos de Entrenamiento (80%) y Pruebas(20%) 
2. Entrenar Modelos a evaluar:
    - XGBoost
    - SVM
    - Ensamble Modelo (RandomForest o Arbol de decisión)
    - Naive Bayes - Gausiano
3. Evaluar los hiperparametros (RandomSearchCV)
4. Entrenar modelos con mejores hiperparámetros encontrados en 3.
5. Evaluar modelos con data de pruebas (Accuracy)

## 1) Lectura de datos - Distribución de conjunto de datos

La distribución del conjunto de datos será de la siguiente manera: 

- Entrenamiento (80%)
- Pruebas (20%)

--- Ya fue aplicado en 2. Pre-procesamiento de datos

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
#Lectura del dataset


#Lectura del dataset

X_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
X_TEST_DF  = pd.read_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv', header=0, sep=",")
Y_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
Y_TEST_DF  = pd.read_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv', header=0, sep=",")

In [3]:
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
]

## 2) Entrenar Modelos a evaluar

In [4]:
## Resultados con la métrica de exactitud (Accuracy) de cada uno de los modelos.
resultados_modelo_metricas_df = pd.DataFrame(columns=['Variable Objetivo','Modelo','Accuracy Train','Accuracy Test', 'Especificidad', "F1_score"])

### 2.1) Random Forest

Entrenamos el modelo de Random Forest de tipo clasificación.

In [5]:
for var_objetivo in var_objetivo_lst:
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    
    rf = RandomForestClassifier(oob_score = True)
    rf.fit(X_train, y_train)

    print('Exactitud del modelo inicial en entrenamiento:', rf.score(X_train, y_train))
    print('Exactitud del modelo inicial en entrenamiento (Out of Bag):', rf.oob_score_)
    print('Exactitud del modelo inicial en validación:', rf.score(X_test, y_test))

    #### Búsqueda aleatoria de mejores hiperparámetros
    # Definición de Grilla
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 10, 11, 12]  # 'auto' equivale a 'sqrt'; None equivale a todas
    max_depth = [int(x) for x in np.linspace(10, 220, num = 11)] + [None]
    min_samples_leaf = [1, 2, 4]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_leaf': min_samples_leaf}

    print('Los valores a probar en la búsqueda aleatoria son:')
    pprint(random_grid)

    print()
    print('Si se probara todas las combinaciones se requeriría entrenar', 
          len(random_grid['n_estimators']) *
          len(random_grid['max_features']) *
          len(random_grid['max_depth']) *
          len(random_grid['min_samples_leaf']),
          'modelos'
          )

    rf = RandomForestClassifier(oob_score=True)
    rf_random = RandomizedSearchCV(estimator = rf, 
                                   param_distributions = random_grid, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    rf_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    rf_random_best = rf_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(rf_random.best_params_)
    print()
    
    
    y_pred = rf_random_best.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'RandomForest' , 
            'Accuracy Train' : rf_random_best.score(X_train, y_train),
            'Accuracy Test' : rf_random_best.score(X_test, y_test),
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print('Exactitud luego de búsqueda aleatoria en entrenamiento:', rf_random_best.score(X_train, y_train))
    print('Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag):', rf_random_best.oob_score_)
    print('Exactitud luego de búsqueda aleatoria en validación:', rf_random_best.score(X_test, y_test))
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)

Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9576342281879194
Exactitud del modelo inicial en validación: 0.9029411764705882
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['auto', 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

Si se probara todas las combinaciones se requeriría entrenar 1440 modelos
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Los hiperparámetros del mejor modelo son:
{'max_depth': 52,
 'max_features': 10,
 'min_samples_leaf': 1,
 'n_estimators': 1600}

****************************************************************************************************
FIBR_PREDS
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 1.0


Exactitud luego de búsqueda aleatoria en entrenamiento: 1.0
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.95875
Exactitud luego de búsqueda aleatoria en validación: 0.9058823529411765
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9838709677419355
F1_score luego de búsqueda aleatoria en validación: 0.8802503785577194
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9890795631825273
Exactitud del modelo inicial en validación: 0.9705882352941176
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['auto', 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

Si se probara todas las combi

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Los hiperparámetros del mejor modelo son:
{'max_depth': 220,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'n_estimators': 1400}

****************************************************************************************************
RAZRIV
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 0.998829953198128
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.9894695787831513
Exactitud luego de búsqueda aleatoria en validación: 0.9676470588235294
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9939577039274925
F1_score luego de búsqueda aleatoria en validación: 0.9575222017057944
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.98225552050

### 2.2) Ensamble - XGBoost

Entrenamos el modelo de XGBoost de tipo clasificación.

In [8]:
from xgboost import XGBClassifier
from pprint import pprint
from sklearn.model_selection import GridSearchCV
import time

## XGBoostRegressor
modelXGBC = XGBClassifier()


## Creamos la Grilla
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
learning_rate = [0.01,0.1]
max_depth = [i for i in range(2,8,2)]
start = time.time()

random_grid = {'max_depth': max_depth,
               'n_estimators': n_estimators,
               'colsample_bytree': [0.2, 0.6, 0.8],
               'min_child_weight': [3, 5, 7],
               'gamma': [0.3, 0.5, 0.7],
               'subsample': [0.4, 0.6, 0.8, 1],
               'learning_rate': learning_rate}

#### Búsqueda aleatoria de mejores hiperparámetros

In [9]:
for var_objetivo in var_objetivo_lst:    
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    xgb_random = RandomizedSearchCV(estimator = modelXGBC, 
                                   param_distributions = random_grid, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    xgb_random.fit(X_train, y_train)
    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    xgb_best_model = xgb_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(xgb_random.best_params_)
    print()


    y_pred = xgb_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_XGBR_train = xgb_best_model.score(X_train, y_train)
    score_XGBR_val = xgb_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'XGBoost' , 
            'Accuracy Train' : score_XGBR_train,
            'Accuracy Test' : score_XGBR_val,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )

    print("*"*100)
    print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_XGBR_train*100:.3f}%")
    print(f"Exactitud luego de búsqueda en grilla en validación: {score_XGBR_val*100:.3f}%")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()
    print(f"{end-start} segundos")

Fitting 3 folds for each of 10 candidates, totalling 30 fits


D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:19:05] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 99.958%
Exactitud luego de búsqueda en grilla en validación: 87.941%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9674267100977199
F1_score luego de búsqueda aleatoria en validación: <function f1_score at 0x000001DE83D5C8B0>
****************************************************************************************************
21.29427385

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:19:19] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 3,
 'n_estimators': 400,
 'subsample': 0.6}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 99.733%
Exactitud luego de búsqueda en grilla en validación: 98.529%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 1.0
F1_score luego de búsqueda aleatoria en validación: <function f1_score at 0x000001DE83D5C8B0>
****************************************************************************************************
33.799187660217285 segundo

...

[03:20:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.01,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 400,
 'subsample': 1}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 88.358%
Exactitud luego de búsqueda en grilla en validación: 78.235%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9840637450199203
F1_score luego de búsqueda aleatoria en validación: <function f1_score at 0x000001DE83D5C8B0>
****************************************************************************************************
133.39063501

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:21:13] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 3,
 'n_estimators': 400,
 'subsample': 0.6}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 99.246%
Exactitud luego de búsqueda en grilla en validación: 91.176%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9778481012658228
F1_score luego de búsqueda aleatoria en validación: <function f1_score at 0x000001DE83D5C8B0>
****************************************************************************************************
148.4271931

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:21:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 300,
 'subsample': 1}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 98.717%
Exactitud luego de búsqueda en grilla en validación: 90.588%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9839743589743589
F1_score luego de búsqueda aleatoria en validación: <function f1_score at 0x000001DE83D5C8B0>
****************************************************************************************************
165.495586633

### 2.3) SVM

Entrenamos el modelo de SVM de tipo clasificación.

In [11]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

modelSVM = svm.SVC()
# Entrenamiento del modelo base - SVM

#### Búsqueda aleatoria de mejores hiperparámetros

In [12]:
for var_objetivo in var_objetivo_lst:
    # Lectura
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    # Definición de grilla
    random_grid = {'kernel': ['rbf'],
                   'C': np.logspace(-4,4,9), # [0.0001, 0.001, ..., 10000]
                   'gamma': np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
                  }

    modelSVM_random = RandomizedSearchCV(estimator = modelSVM, 
                                   param_distributions = random_grid, 
                                   cv = 12,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    modelSVM_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    modelSVM_best_model = modelSVM_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(modelSVM_random.best_params_)
    print()


    
    
    y_pred = modelSVM_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_SVM_train = modelSVM_best_model.score(X_train, y_train)
    score_SVM_test = modelSVM_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'SVM' , 
            'Accuracy Train' : score_SVM_train,
            'Accuracy Test' : score_SVM_test,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    
    print("*"*100)

    print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_SVM_train:.3f}")
    print(f"Exactitud luego de búsqueda en grilla en validación: {score_SVM_test:.3f}")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()
    print(end-start)

Fitting 12 folds for each of 10 candidates, totalling 120 fits
Los hiperparámetros del mejor modelo son:
{'C': 10000.0, 'gamma': 0.01, 'kernel': 'rbf'}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 1.000
Exactitud luego de búsqueda en grilla en validación: 0.812
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.8827361563517915
F1_score luego de búsqueda aleatoria en validación: 0.8206971094565735
****************************************************************************************************
352.1364185810089
Fitting 12 folds for each of 10 candidates, totalling 120 fits
Los hiperparámetros del mejor modelo son:
{'C': 10000.0, 'gamma': 0.01, 'kernel': 'rbf'}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento

### 2.4. Naive Bayes

Entrenamos el modelo de Naive Bayes (Gausiano) de tipo clasificación.

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
modelNB = GaussianNB()
#modelNB.fit(X_train, y_train)

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(params_NB)


for var_objetivo in var_objetivo_lst:
    # Lectura
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)


    modelNB_random = RandomizedSearchCV(estimator = modelNB, 
                                   param_distributions=params_NB, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    modelNB_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    modelNB_best_model = modelNB_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(modelNB_random.best_params_)
    print()


    
    
    
    y_pred = modelNB_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_NB_train = modelNB_best_model.score(X_train, y_train)
    score_NB_test = modelNB_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'Naive Bayes - Gaussian' , 
            'Accuracy Train' : score_NB_train,
            'Accuracy Test' : score_NB_test,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    print("*"*100)


    print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_NB_train:.3f}")
    print(f"Exactitud luego de búsqueda en grilla en validación: {score_NB_test:.3f}")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()

Los valores a probar en la búsqueda aleatoria son:
{'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31012970e-02, 1.87381742e-02,
       1.51991108e-02, 1.23284674e-02, 1.00000000e-02, 8.11130831e-03,
       6.57933225e-03, 5.33669923e-03, 4.32876128e-03, 3.51119173e-03,
       2.84803587e-03, 2.31012970e-03, 1.87381742e-03, 1.51991108e-03,
       1.23284674e-03, 1.00000000e-03, 8.11130831e-04, 6.57933225e-04,
       5.33669923e-04, 4.32876128e-04, 3.51119173e-04, 2.84803587e-04,
       2.31012970e-04, 1.87381742e-04, 1.51991108e-04, 1.23284674e-04,
       1.00000000e-04, 8.11130831e-05, 6.57933225e-05, 5.33669923e-05,
       4.32876128e-05, 3.51119173e-05, 2.84803587e-05, 2.31012970e-05,
       1

Los hiperparámetros del mejor modelo son:
{'var_smoothing': 0.03511191734215131}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 0.702
Exactitud luego de búsqueda en grilla en validación: 0.485
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.46474358974358976
F1_score luego de búsqueda aleatoria en validación: 0.5876174926084464
****************************************************************************************************


In [15]:
resultados_modelo_metricas_df

,Variable Objetivo,Modelo,Accuracy Train,Accuracy Test,Especificidad,F1_score
0,FIBR_PREDS,RandomForest,1.000000,0.897059,0.983713,0.867688
1,PREDS_TAH,RandomForest,0.998474,0.985294,1.000000,0.977996
2,JELUD_TAH,RandomForest,0.998451,0.979412,1.000000,0.969225
3,FIBR_JELUD,RandomForest,1.000000,0.944118,0.996894,0.919836
4,A_V_BLOK,RandomForest,1.000000,0.970588,1.000000,0.958553
5,OTEK_LANC,RandomForest,1.000000,0.905882,0.983871,0.880250
6,RAZRIV,RandomForest,0.998830,0.967647,0.993958,0.957522
7,DRESSLER,RandomForest,1.000000,0.950000,0.996914,0.928507
8,ZSN,RandomForest,1.000000,0.785294,0.968127,0.745669
9,REC_IM,RandomForest,1.000000,0.920588,0.990506,0.890983


In [63]:
import tensorflow as tf

print(tf.__version__)


# Lectura
X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]=="FIBR_PREDS"].dropna(axis='columns')
y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]=="FIBR_PREDS", "FIBR_PREDS"]
X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]=="FIBR_PREDS"].dropna(axis='columns')
y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]=="FIBR_PREDS", "FIBR_PREDS"]
X_train.drop(["var_objetivo"], axis=1, inplace=True)
X_test.drop(["var_objetivo"], axis=1, inplace=True)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.iloc[0].shape)),
    tf.keras.layers.Dense(600, activation=tf.nn.relu),
    tf.keras.layers.Dense(80, activation=tf.nn.relu),
    tf.keras.layers.Dense(60, activation=tf.nn.softmax)
])

2.8.2


In [64]:
X_train.shape

(2384, 66)

In [65]:
y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2379    1.0
2380    1.0
2381    1.0
2382    1.0
2383    1.0
Name: FIBR_PREDS, Length: 2384, dtype: float64

In [66]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, validation_split=0.15, batch_size=128)

Epoch 1/30
16/16 [==============================] - 0s 10ms/step - loss: 2.4366 - accuracy: 0.5306 - val_loss: 1.3153 - val_accuracy: 0.0000e+00
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6467 - accuracy: 0.6604 - val_loss: 0.7963 - val_accuracy: 0.2346
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.5602 - accuracy: 0.7192 - val_loss: 0.6887 - val_accuracy: 0.5112
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.5079 - accuracy: 0.7463 - val_loss: 0.6837 - val_accuracy: 0.5447
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.4709 - accuracy: 0.7606 - val_loss: 0.4810 - val_accuracy: 0.8045
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.4296 - accuracy: 0.8006 - val_loss: 0.5864 - val_accuracy: 0.7179
Epoch 7/30
16/16 [==============================] - 0s 3ms/step - loss: 0.4004 - accuracy: 0.8169 - val_loss: 0.5957 - val_accuracy: 0.6983
Epoch 8/30
16/1

In [67]:
model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 1ms/step - loss: 0.6749 - accuracy: 0.8588


[0.6749401688575745, 0.8588235378265381]

In [ ]:
ll